<a href="https://colab.research.google.com/github/Ksrinivas286/Fmml/blob/main/Copy_of_Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [2]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [3]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [4]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [5]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [6]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [7]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [8]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [9]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [10]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [11]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [12]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [13]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [14]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


Yes, averaging the validation accuracy across multiple splits of your dataset can often give you more consistent and reliable results compared to relying on a single split. This technique is commonly known as "cross-validation."

Here's why cross-validation can be beneficial for evaluating machine learning models:

1. **Reduced Variance**: Averaging results from multiple splits helps reduce the impact of randomness or variability in the data. It provides a more stable estimate of model performance.

2. **Better Generalization**: By training and evaluating the model on different subsets of the data, you get a better sense of how well your model generalizes to unseen data. It can help you detect overfitting or underfitting.

3. **More Robust Evaluation**: It minimizes the risk of getting lucky or unlucky with a single data split. If your dataset has some unusual patterns or outliers, these might disproportionately affect a single split but not the overall average.

Common cross-validation techniques include k-fold cross-validation and stratified k-fold cross-validation. These methods divide your dataset into multiple subsets (folds), train and evaluate the model multiple times, and then average the results.

However, keep in mind that cross-validation can be more computationally expensive, as it involves training and evaluating the model multiple times. The choice of the number of folds (e.g., 5-fold, 10-fold) and the specific technique depends on your dataset and the problem you're trying to solve.



2.Cross-validation, while providing a more reliable estimate of model performance compared to a single split, doesn't directly estimate the test accuracy on unseen, real-world data. Instead, it gives you a more accurate estimate of how your model might perform on data from the same distribution as your training data.

Here's the key distinction:

1. **Validation Accuracy**: Cross-validation estimates how well your model generalizes to data that is similar to your training data but not part of it. It provides a useful evaluation metric for model selection and hyperparameter tuning.

2. **Test Accuracy**: The test accuracy, on the other hand, measures how well your model performs on completely unseen, real-world data. It is a crucial indicator of your model's performance in a production or deployment environment.

Cross-validation helps you make informed decisions during model development, such as choosing between different models or selecting optimal hyperparameters. It can give you confidence that your model is likely to perform well on unseen data from the same distribution. However, the true test accuracy can only be determined by evaluating your model on a separate, held-out test dataset that was not used during training or cross-validation.

In summary, while cross-validation provides a more accurate estimate of how your model might perform on similar data, it doesn't replace the need for a final evaluation on a true test dataset to assess real-world performance accurately.



3.The number of iterations (or folds) in cross-validation can have an impact on the estimate of model performance. In general, using a higher number of iterations can provide a more accurate and reliable estimate, but there are trade-offs to consider:

1. **More Stable Estimate**: Increasing the number of iterations reduces the impact of randomness in the data split, which can lead to a more stable estimate of model performance. With a larger number of folds, you get a better average over different subsets of the data.

2. **Reduced Variance**: Higher iterations tend to reduce the variance in the estimated performance metrics. This means that the estimate is less sensitive to the specific random split of data, making it more reliable.

However, there are diminishing returns when it comes to the number of iterations:

1. **Computational Cost**: As you increase the number of iterations, the computational cost of cross-validation also increases. Training and evaluating the model multiple times can be time-consuming, especially if your dataset is large or your model is complex.

2. **Bias-Variance Trade-off**: With too many iterations (e.g., leave-one-out cross-validation, where each data point is a fold), you might introduce bias into the estimate because each fold becomes very similar to the entire dataset. This can lead to underestimating the model's performance on truly unseen data.

3. **Sample Size**: The choice of the number of iterations should also take into account your dataset's size. If your dataset is small, you might need to use a smaller number of iterations to ensure each fold still contains a reasonable amount of data.

In practice, common choices for the number of iterations include 5-fold, 10-fold, or even 3-fold cross-validation. The choice should strike a balance between obtaining a reliable estimate and managing computational costs. It's often a good idea to experiment with different numbers of iterations to see how the estimate stabilizes as you increase them, and choose an appropriate value based on your specific dataset and computational resources.



4.Increasing the number of iterations in cross-validation can help mitigate some of the issues associated with small training or validation datasets to some extent. However, it doesn't completely solve these issues, and there are limitations to what increasing iterations can achieve:

1. **Mitigating Variability**: With a small training dataset, there is a higher risk of the training data not being representative of the overall data distribution. Increasing the number of iterations (e.g., using k-fold cross-validation) can help by repeatedly shuffling and splitting the small dataset, providing a more comprehensive view of the data.

2. **Reducing Overfitting**: In cases of small datasets, models may overfit the training data. Cross-validation can help identify overfitting by repeatedly assessing model performance on different subsets of the data. However, more iterations won't fundamentally address the overfitting problem; it's crucial to choose an appropriate model complexity and regularization techniques.

3. **Limited Data**: If you have an extremely small dataset, increasing iterations may lead to very small training or validation subsets in each fold, which can introduce instability in the performance estimates. In such cases, the estimates may not be reliable, and it's essential to collect more data if possible.

4. **Computational Overhead**: While increasing iterations can improve stability, it also comes with computational costs. If your dataset is very small, you might find that you're spending more computational resources on cross-validation than necessary.

In summary, increasing the number of iterations in cross-validation can be a helpful strategy to make the best use of a small dataset, but it's not a complete solution. It's essential to be aware of the limitations of your dataset size and consider other techniques like data augmentation, feature engineering, or even collecting more data to improve model performance and reliability. Additionally, when dealing with very small datasets, models with fewer parameters or more robust regularization can be beneficial.